In [11]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, Trainer
from datasets import Dataset, ClassLabel

In [12]:
from train_test_split import select_eval
from preprocessing import preprocess
from evaluation import evaluate
from bert import tokenize, get_BERT, compute_metrics, prepare_dataset

In [13]:
logging.basicConfig(level=logging.INFO)

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [15]:
MODEL = 'models/roberta-baseline'
TOKENIZER = 'cardiffnlp/twitter-roberta-base-sentiment-latest' # 'bert-base-uncased'
PREPROCESSING = None

In [16]:
df_eval = select_eval()
dataset_eval = prepare_dataset(df_eval, preprocessing=PREPROCESSING)

Casting the dataset: 100%|██████████| 4/4 [00:00<00:00, 19.02ba/s]


In [17]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
eval_tokenized = tokenize(dataset_eval, tokenizer)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest/resolve/main/config.json from cache at /home/jonhue/.cache/huggingface/transformers/c26252806565e705085b65f69d7d544c05112fee06744845d6c067efcb278fff.675a397541d62bd9db2f771a42252113ba14c2589f847d4a1a49fbb504f72bbd
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Negative": 0,
    "Neutral": 1,
    "Positive"

In [18]:
model = get_BERT(MODEL, device)

loading configuration file models/roberta-baseline/config.json
Model config RobertaConfig {
  "_name_or_path": "models/roberta-baseline",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file models/roberta-baseline/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaF

In [19]:
trainer = Trainer(model, eval_dataset=eval_tokenized, tokenizer=tokenizer, compute_metrics=compute_metrics)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: index, text. If index, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40000
  Batch size = 8


INFO:root:---
* accuracy: 0.896325
* precision: 0.8912657728706624
* recall: 0.9030614792988064
* f1: 0.8971248542581429
---
INFO:root:---
* bce: 0.2491948601735341
* auc: 0.9622035612642098
---


{'eval_loss': 0.24919487535953522,
 'eval_accuracy': 0.896325,
 'eval_precision': 0.8912657728706624,
 'eval_recall': 0.9030614792988064,
 'eval_f1': 0.8971248542581429,
 'eval_bce': 0.2491948601735341,
 'eval_auc': 0.9622035612642098,
 'eval_confidence': 0.9075518846511841,
 'eval_confidence_std': 0.12932221591472626,
 'eval_correct_confidence': 0.9262843728065491,
 'eval_correct_confidence_std': 0.11325863003730774,
 'eval_incorrect_confidence': 0.745598316192627,
 'eval_incorrect_confidence_std': 0.14543266594409943,
 'eval_runtime': 187.6014,
 'eval_samples_per_second': 213.218,
 'eval_steps_per_second': 26.652}